# Creating a frequency finder auto-encoder

## Import Libraries

In [1]:
import tensorflow as tf
import numpy as np
import random
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from tensorflow.keras import callbacks
import tensorflow.keras as keras
from tensorflow.keras import layers
import umap
from babyplots import Babyplot
#import hdbscan
from matplotlib import pyplot
from models import *
from converters import *
from helpers import *
from jupyterthemes import jtplot
jtplot.style(theme='monokai', context='notebook', ticks=True, grid=False)

2021-10-06 11:05:00.177405: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2021-10-06 11:05:00.177461: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
%load_ext autoreload
%autoreload 2

## Get drive path 

In [3]:
drive_path = find_drive_path()
drive_path

UnboundLocalError: local variable 'path' referenced before assignment

## Load Data from drive

In [4]:
data = pd.read_pickle(f"{drive_path}/Data/Inputs/Synthetisch/Syn_tot.pkl")
data.head()

,IID,Pheno,Geno,State
0,0pop1,1,"[2, 2, 2, 2, 2, 0, 2, 2, 2, 2, 2, 2, 2, 2, 0, ...",pop1sick
1,1pop1,1,"[2, 2, 2, 2, 2, 2, 2, 1, 2, 2, 2, 2, 2, 1, 1, ...",pop1sick
2,2pop1,1,"[2, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 0, 2, ...",pop1sick
3,3pop1,0,"[2, 2, 1, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, ...",pop1healthy
4,4pop1,1,"[2, 2, 2, 2, 2, 0, 2, 2, 2, 2, 2, 2, 2, 1, 2, ...",pop1sick


In [5]:
data.tail()

,IID,Pheno,Geno,State
1995,995pop2,0,"[2, 0, 0, 0, 1, 2, 2, 2, 1, 0, 1, 0, 2, 1, 1, ...",pop2healthy
1996,996pop2,0,"[1, 1, 0, 2, 0, 1, 1, 0, 0, 0, 0, 0, 2, 1, 0, ...",pop2healthy
1997,997pop2,0,"[1, 1, 0, 0, 2, 1, 0, 0, 0, 0, 1, 1, 2, 1, 2, ...",pop2healthy
1998,998pop2,0,"[2, 2, 1, 2, 0, 1, 0, 2, 0, 1, 0, 2, 2, 0, 0, ...",pop2healthy
1999,999pop2,0,"[1, 0, 1, 1, 2, 1, 1, 0, 0, 0, 2, 1, 1, 1, 1, ...",pop2healthy


## Preprocessing data into tensors 

### Keeping only the minor alleles as 1

In [6]:
X = only_minor(data["Geno"])
X

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 1., 1., 0.],
       [0., 0., 0., ..., 1., 0., 1.],
       ...,
       [0., 0., 1., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.]])

### Train test split

In [7]:
split = splitter([X],0.1)
[X_train, X_test] = tensor_converter(split)
X = tensor_converter(X)
X = tf.stack(X, axis = 0)

2021-10-01 16:03:49.441455: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-10-01 16:03:49.442090: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2021-10-01 16:03:49.442207: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublas.so.11'; dlerror: libcublas.so.11: cannot open shared object file: No such file or directory
2021-10-01 16:03:49.442304: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublasLt.so.11'; dlerror: libcublasLt.so.11: cannot open shared object file: No such file or directory
2021-10-01 16:03:49.444474: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Co

## Loading hyper-parameters

In [24]:
BATCH_SIZE = 64
WEIGHT_DECAY = 0.001
PRUNING = 0.5
LEARNING_RATE = 0.01
first_layer = 10

In [25]:
input_len = int(tf.shape(X_train)[1])
ff = frequencyfinder(input_len, first_layer)
ff.build((None, input_len))

In [26]:
output_test = ff(X)

In [27]:
output_test[0]

<tf.Tensor: shape=(10000,), dtype=float32, numpy=
array([9.9231533e-05, 9.9231533e-05, 1.0053032e-04, ..., 9.9231525e-05,
       1.0233781e-04, 1.0048799e-04], dtype=float32)>

In [29]:
yes = tfpl.IndependentLogistic(output_test[0])

In [10]:
from tensorflow_probability.python import layers as tfpl

In [32]:
dir(yes)

['_TF_MODULE_IGNORED_PROPERTIES',
 '__call__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setstate__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_activity_regularizer',
 '_add_trackable',
 '_add_variable_with_custom_getter',
 '_already_warned',
 '_auto_track_sub_layers',
 '_autocast',
 '_autographed_call',
 '_build_input_shape',
 '_call_accepts_kwargs',
 '_call_arg_was_passed',
 '_call_fn_arg_defaults',
 '_call_fn_arg_positions',
 '_call_fn_args',
 '_call_full_argspec',
 '_callable_losses',
 '_cast_single_input',
 '_check_variables',
 '_checkpoint_dependencies',
 '_clear_losses',
 '_compute_dtype',
 '_compute_dtype_object',
 '_convert_to_tensor_fn',
 '_dedup_weights',
 '_default_trai

In [38]:
x = tf.constant(3.0)
with tf.GradientTape() as g:
    g.watch(x)
    y = x*x
    dy_dx = g.gradient(y,x)
print(dy_dx)

tf.Tensor(6.0, shape=(), dtype=float32)
